In [90]:
import folium
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
import alphashape
import geopandas as gpd

In [91]:
fmap = folium.Map(location=[37.2633325, 127.0287472], tiles="Stamen Toner", zoom_start=17, width=800, height=600)
# tiles="cartodbpositron"

folium.Marker([37.2633325, 127.0287472], tooltip="Suwon City Hall", icon=folium.Icon(color='red', icon='star')).add_to(fmap)

In [92]:
"""
Randomly generated geopoint (http://www.geomidpoint.com/random/)
	# of points : 100
	starting point : Suwon City Hall
	within 1km distance (circular region)
"""
data = pd.read_csv("./data/random.csv", usecols=['latitude', 'longitude'])

d_list = data.values.tolist()
# print(d_list[0])
# print(data)

In [93]:
fire_center = d_list[len(d_list) - 1]
print(fire_center)
folium.Circle(fire_center, tooltip='fire', radius=300, color='red', fill_color='red').add_to(fmap)
# radius in meters

for i in range (0, len(d_list) - 1):
	folium.Circle(d_list[i], tooltip=str(i), radius=2).add_to(fmap)

fmap

[37.26549784, 127.03400625]


In [94]:
def haversine_distance(point, center):
	lat1, lon1, lat2, lon2 = map(radians, [point[0], point[1], center[0], center[1]])
	del_lat = lat1 - lat2
	del_lon = lon1 - lon2

	a = sin(del_lat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(del_lon / 2) **2
	c = 2 * asin(sqrt(a))
	r = 6371 # radius of earth in km
	return c * r * 1000 #return m

In [95]:
def get_inside(d_list, center, radius):
	inside_id = [i for i in range(0, len(d_list)) if haversine_distance(d_list[i], center) <= radius]
	inside_points = [d_list[i] for i in inside_id]

	points = gpd.GeoDataFrame(inside_points)

	return points

In [98]:
points = get_inside(d_list, fire_center, 300)
points

,0,1
0,37.267077,127.031659
1,37.267562,127.032671
2,37.267251,127.032994
3,37.263056,127.033974
4,37.262984,127.034919
5,37.267065,127.033773
6,37.264900,127.031563
7,37.264222,127.033400
8,37.267851,127.033051
9,37.264003,127.034552


In [99]:
hull = alphashape.alphashape(points)

KeyError: 'geometry'